<a href="https://colab.research.google.com/github/SampleBias/open_immune/blob/main/open_immune_LLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Step 1. Install Python Packages

In [1]:
# Installs Unsloth, Xformers (Flash Attention) and all other packages!
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps "xformers<0.0.27" "trl<0.9.0" peft accelerate bitsandbytes
!pip install torch --upgrade --force-reinstall

  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-qcpoy0f9/unsloth_2db78104bc2a4345b5a2cfecf15123cc
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-qcpoy0f9/unsloth_2db78104bc2a4345b5a2cfecf15123cc
  Resolved https://github.com/unslothai/unsloth.git to commit 0f2e484f3931d1a558dc3a5967c8da665a2e7252
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.4/103.4 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.3/9.3 MB 68.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 51.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 75.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 45.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 20.0 MB/s eta

Step 2. Import Python Packages

In [2]:
from unsloth import FastLanguageModel

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


In [3]:
import torch
import os
import json
import pandas as pd
from datasets import Dataset, DatasetDict
from datasets import load_dataset
from huggingface_hub import notebook_login
from transformers import TrainingArguments
import subprocess
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

Step 3. Login to Your Hugging Face with hf_token. (write access token)

In [4]:
notebook_login()

Step 4. Convert your JSON dataset to Llama3 finetuning format

In [5]:
import json
import pandas as pd
import os
from datasets import Dataset, DatasetDict

class LlamaInstructDataset:
    def __init__(self, data):
        self.data = data
        self.prompts = []
        self.create_prompts()

    def create_prompt(self, row):
        try:
            prompt = f"""[begin_of_text][start_header_id]system:[end_header_id]{row['instruction']}[eot_id][start_header_id]user:[end_header_id]{row['input']}[eot_id][start_header_id]assistant:[end_header_id]{row['output']}[eot_id]"""
            return prompt
        except KeyError as e:
            print(f"Warning: Skipping row due to missing key: {e}")
            return None

    def create_prompts(self):
        for row in self.data:
            prompt = self.create_prompt(row)
            if prompt:
                self.prompts.append(prompt)

    def get_dataset(self):
        df = pd.DataFrame({'prompt': self.prompts})
        return df

def create_dataset_hf(dataset):
    dataset.reset_index(drop=True, inplace=True)
    hf_dataset = Dataset.from_pandas(dataset)
    return DatasetDict({"train": hf_dataset})

if __name__ == "__main__":
    huggingface_user = "SampleBias"
    dataset_name = "Fine_Tune_Data_Open_Immune_V1"

    try:
        with open('/content/Fine_Tune_Data_Open_Immune_Mini.json', 'r') as f:
            data = json.load(f)
    except json.JSONDecodeError as e:
        print(f"Error decoding JSON: {e}")
        print("Attempting to read file line by line...")
        data = []
        with open('/content/Fine_Tune_Data_Open_Immune_Mini.json', 'r') as f:
            for line in f:
                try:
                    data.append(json.loads(line.strip()))
                except json.JSONDecodeError:
                    print(f"Warning: Skipping invalid JSON line")

    dataset = LlamaInstructDataset(data)
    df = dataset.get_dataset()

    processed_data_path = "/processed_data"
    os.makedirs(processed_data_path, exist_ok=True)

    try:
        llama2_dataset = create_dataset_hf(df)
        llama2_dataset.save_to_disk(os.path.join(processed_data_path, "llama2_dataset"))
        llama2_dataset.push_to_hub(f"{huggingface_user}/{dataset_name}")
        print(f"Dataset successfully created and pushed to {huggingface_user}/{dataset_name}")
    except Exception as e:
        print(f"Error creating or saving dataset: {e}")
        print("Dumping data to CSV as fallback")
        csv_path = os.path.join(processed_data_path, "llama2_dataset.csv")
        df.to_csv(csv_path, index=False)
        print(f"Data saved to {csv_path}")

Saving the dataset (0/1 shards):   0%|          | 0/607 [00:00<?, ? examples/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Dataset successfully created and pushed to SampleBias/Fine_Tune_Data_Open_Immune_V1


Step 5. LoRa Finetuning Configurations "finetuned_model" sets your models name on HF

"num_train_epochs" sets the number of epochs for training

(epoch = 1 pass through your entire dataset)

In [6]:
# Defining the configuration for the base model, LoRA and training
config = {
    "hugging_face_username":huggingface_user,
    "model_config": {
        "base_model":"unsloth/llama-3-8b-Instruct-bnb-4bit", # The base model
        "finetuned_model":"llama-3-8b-Instruct-bnb-4bit-open_immune_V1", # The finetuned model
        "max_seq_length": 2048, # The maximum sequence length
        "dtype":torch.float16, # The data type
        "load_in_4bit": True, # Load the model in 4-bit
    },
    "lora_config": {
      "r": 16, # The number of LoRA layers 8, 16, 32, 64
      "target_modules": ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj"], # The target modules
      "lora_alpha":16, # The alpha value for LoRA
      "lora_dropout":0, # The dropout value for LoRA
      "bias":"none", # The bias for LoRA
      "use_gradient_checkpointing":True, # Use gradient checkpointing
      "use_rslora":False, # Use RSLora
      "use_dora":False, # Use DoRa
      "loftq_config":None # The LoFTQ configuration
    },
    "training_dataset":{
        "name":f"{huggingface_user}/{dataset_name}", # The dataset name(huggingface/datasets)
        "split":"train", # The dataset split
        "input_field":"prompt", # The input field
    },
    "training_config": {
        "per_device_train_batch_size": 2, # The batch size
        "gradient_accumulation_steps": 4, # The gradient accumulation steps
        "warmup_steps": 5, # The warmup steps
        "max_steps":0, # The maximum steps (0 if the epochs are defined)
        "num_train_epochs": 5, # The number of training epochs(0 if the maximum steps are defined)
        "learning_rate": 2e-4, # The learning rate
        "fp16": not torch.cuda.is_bf16_supported(),  # The fp16
        "bf16": torch.cuda.is_bf16_supported(), # The bf16
        "logging_steps": 1, # The logging steps
        "optim" :"adamw_8bit", # The optimizer
        "weight_decay" : 0.01,  # The weight decay
        "lr_scheduler_type": "linear", # The learning rate scheduler
        "seed" : 42, # The seed
        "output_dir" : "outputs", # The output directory
    }
}

Step 6. Load Llama3-8B, QLoRA & Trainer Model

In [7]:
# Loading the model and the tokinizer for the model
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = config.get("model_config").get("base_model"),
    max_seq_length = config.get("model_config").get("max_seq_length"),
    dtype = config.get("model_config").get("dtype"),
    load_in_4bit = config.get("model_config").get("load_in_4bit"),
)

# Setup for QLoRA/LoRA peft of the base model
model = FastLanguageModel.get_peft_model(
    model,
    r = config.get("lora_config").get("r"),
    target_modules = config.get("lora_config").get("target_modules"),
    lora_alpha = config.get("lora_config").get("lora_alpha"),
    lora_dropout = config.get("lora_config").get("lora_dropout"),
    bias = config.get("lora_config").get("bias"),
    use_gradient_checkpointing = config.get("lora_config").get("use_gradient_checkpointing"),
    random_state = 42,
    use_rslora = config.get("lora_config").get("use_rslora"),
    use_dora = config.get("lora_config").get("use_dora"),
    loftq_config = config.get("lora_config").get("loftq_config"),
)

# Loading the training dataset
dataset_train = load_dataset(config.get("training_dataset").get("name"), split = config.get("training_dataset").get("split"))

# Setting up the trainer for the model
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset_train,
    dataset_text_field = config.get("training_dataset").get("input_field"),
    max_seq_length = config.get("model_config").get("max_seq_length"),
    dataset_num_proc = 2,
    packing = False,
    args = TrainingArguments(
        per_device_train_batch_size = config.get("training_config").get("per_device_train_batch_size"),
        gradient_accumulation_steps = config.get("training_config").get("gradient_accumulation_steps"),
        warmup_steps = config.get("training_config").get("warmup_steps"),
        max_steps = config.get("training_config").get("max_steps"),
        num_train_epochs= config.get("training_config").get("num_train_epochs"),
        learning_rate = config.get("training_config").get("learning_rate"),
        fp16 = config.get("training_config").get("fp16"),
        bf16 = config.get("training_config").get("bf16"),
        logging_steps = config.get("training_config").get("logging_steps"),
        optim = config.get("training_config").get("optim"),
        weight_decay = config.get("training_config").get("weight_decay"),
        lr_scheduler_type = config.get("training_config").get("lr_scheduler_type"),
        seed = 42,
        output_dir = config.get("training_config").get("output_dir"),
    ),
)

config.json:   0%|          | 0.00/1.15k [00:00<?, ?B/s]

==((====))==  Unsloth: Fast Llama patching release 2024.7
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.3.1+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.26.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/131 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/51.1k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/459 [00:00<?, ?B/s]

Unsloth 2024.7 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


Generating train split:   0%|          | 0/607 [00:00<?, ? examples/s]

Map (num_proc=2):   0%|          | 0/607 [00:00<?, ? examples/s]

In [8]:
#@title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla T4. Max memory = 14.748 GB.
5.594 GB of memory reserved.


Step 7. Train Your Finetuned Model

In [9]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 607 | Num Epochs = 5
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 380
 "-____-"     Number of trainable parameters = 41,943,040


Step,Training Loss
1,2.853600
2,2.907000
3,2.925800
4,2.886800
5,2.726400
6,2.813300
7,2.184400
8,2.339300
9,2.556900
10,2.118900


Step,Training Loss
1,2.853600
2,2.907000
3,2.925800
4,2.886800
5,2.726400
6,2.813300
7,2.184400
8,2.339300
9,2.556900
10,2.118900


Step 8. Save Trainer Stats

In [10]:
with open("trainer_stats.json", "w") as f:
    json.dump(trainer_stats, f, indent=4)

Step 9. Save Finetuned Model & Push to HF Hub

In [11]:
!git clone --recursive https://github.com/ggerganov/llama.cpp
%cd llama.cpp
!make clean && make all -j
%cd ..

Cloning into 'llama.cpp'...
remote: Enumerating objects: 29676, done.
remote: Counting objects: 100% (9396/9396), done.
remote: Compressing objects: 100% (434/434), done.
remote: Total 29676 (delta 9198), reused 8985 (delta 8959), pack-reused 20280
Receiving objects: 100% (29676/29676), 51.05 MiB | 22.96 MiB/s, done.
Resolving deltas: 100% (21271/21271), done.
Submodule 'kompute' (https://github.com/nomic-ai/kompute.git) registered for path 'ggml/src/kompute'
Cloning into '/content/llama.cpp/ggml/src/kompute'...
remote: Enumerating objects: 9090, done.        
remote: Counting objects: 100% (225/225), done.        
remote: Compressing objects: 100% (137/137), done.        
remote: Total 9090 (delta 99), reused 172 (delta 78), pack-reused 8865        
Receiving objects: 100% (9090/9090), 17.58 MiB | 14.93 MiB/s, done.
Resolving deltas: 100% (5706/5706), done.
Submodule path 'ggml/src/kompute': checked out '4565194ed7c32d1d2efa32ceab4d3c6cae006306'
/content/llama.cpp
I ccache not found. 

In [12]:
model.save_pretrained_gguf(config.get("model_config").get("finetuned_model"), tokenizer, quantization_method = "q4_k_m")
model.push_to_hub_gguf(config.get("model_config").get("finetuned_model"), tokenizer, quantization_method = "q4_k_m")

Unsloth: Kaggle/Colab has limited disk space. We need to delete the downloaded
model which will save 4-16GB of disk space, allowing you to save on Kaggle/Colab.
Unsloth: Will remove a cached repo with size 5.7G


Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 36.36 out of 50.99 RAM for saving.


100%|██████████| 32/32 [00:20<00:00,  1.54it/s]


Unsloth: Saving tokenizer... Done.
Unsloth: Saving model... This might take 5 minutes for Llama-7b...
Done.


Unsloth: Converting llama model. Can use fast conversion = False.


...

Unsloth: Saving tokenizer... Done.
Unsloth: Saving model... This might take 5 minutes for Llama-7b...
Done.
==((====))==  Unsloth: Conversion from QLoRA to GGUF information
   \\   /|    [0] Installing llama.cpp will take 3 minutes.
O^O/ \_/ \    [1] Converting HF to GGUF 16bits will take 3 minutes.
\        /    [2] Converting GGUF 16bits to ['q4_k_m'] will take 10 minutes each.
 "-____-"     In total, you will have to wait at least 16 minutes.

Unsloth: [0] Installing llama.cpp. This will take 3 minutes...
Unsloth: [1] Converting model at llama-3-8b-Instruct-bnb-4bit-open_immune_V1 into f16 GGUF format.
The output location will be ./llama-3-8b-Instruct-bnb-4bit-open_immune_V1/unsloth.F16.gguf
This will take 3 minutes...
INFO:hf-to-gguf:Loading model: llama-3-8b-Instruct-bnb-4bit-open_immune_V1
INFO:gguf.gguf_writer:gguf: This GGUF file is for Little Endian only
INFO:hf-to-gguf:Set model parameters
INFO:hf-to-gguf:gguf: context length = 8192
INFO:hf-to-gguf:gguf: embedding length = 40

unsloth.F16.gguf:   0%|          | 0.00/16.1G [00:00<?, ?B/s]

Saved GGUF to https://huggingface.co/SampleBias/llama-3-8b-Instruct-bnb-4bit-open_immune_V1
Unsloth: Uploading GGUF to Huggingface Hub...


unsloth.Q4_K_M.gguf:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

Saved GGUF to https://huggingface.co/SampleBias/llama-3-8b-Instruct-bnb-4bit-open_immune_V1
